## env

In [1]:
!nvidia-smi -L
!nvcc -V
!nvidia-smi

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-1242830d-0dff-bf8c-ac33-61a08f8c447a)
GPU 1: NVIDIA A100-SXM4-40GB (UUID: GPU-7d81048e-a999-9a44-bacb-6cfa0327cc98)
GPU 2: NVIDIA A100-SXM4-40GB (UUID: GPU-77e505f3-e454-9468-bf4c-a21cf5e7dac6)
GPU 3: NVIDIA A100-SXM4-40GB (UUID: GPU-b3a0be24-9fd6-96c5-f6df-a78b70e2096a)
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Fri Jul 29 18:18:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|========

In [2]:
import torch

print(torch.__version__)

/users/k21116947/miniconda3/envs/Mnet/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.12.0


In [3]:
import os
# os.environ["TF_ENABLE_ONEDNN_OPTS"]=0
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import sys
sys.path.append('Code/code/')
from load_data import load_MEG_dataset
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
from torch.cuda import amp
import torch.nn.functional as F
import math
# %env CUBLAS_WORKSPACE_CONFIG=:16:8
from scipy.integrate import simps
from mne.time_frequency import psd_array_welch
from band_power import (
    bandpower_multi_bands,
    standard_scaling_sklearn,
)
import sklearn
from sklearn import preprocessing

2022-07-29 18:18:48.673729: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
device = torch.device('cuda' if  torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Mnet

## Load

In [5]:
Split = 0.90
X_train, y_train = load_MEG_dataset([str(i).zfill(3) for i in range(1,2)], mode = 'concatenate', output_format='numpy',shuffle = False, training=True, train_test_split=Split, batch_size=500)#, pca_n_components =30)
X_test, y_test = load_MEG_dataset([str(i).zfill(3) for i in range(1,2)], mode = 'concatenate', output_format='numpy',shuffle = False, training=False, train_test_split=Split, batch_size=500)#, pca_n_components =30)

X_train, X_test = (X_train-X_train.mean())/X_train.std(), (X_test-X_test.mean())/X_test.std()

X_train = X_train[:, None, ...]
X_test = X_test[:, None, ...]

# X_train=np.repeat(X_train,3,axis=1)
# X_test=np.repeat(X_test,3,axis=1)


# X_train = scipy.interpolate.SmoothSphereBivariateSpline()

y_train = (y_train / 2) - 1
y_test = (y_test / 2) - 1

X_train_tensors = torch.Tensor(X_train)
X_test_tensors = torch.Tensor(X_test)
y_train_tensors = torch.from_numpy(y_train) 
y_test_tensors = torch.from_numpy(y_test)

# y_train_tensors = F.one_hot(y_train_tensors)
# y_test_tensors = F.one_hot(y_test_tensors)
# X_train_tensors = F.interpolate(X_train_tensors, size=(272, 272), mode ='bicubic')
# X_test_tensors = F.interpolate(X_test_tensors, size=(272, 272), mode ='bicubic')

Loading subject 001
Data loaded
Subject 001 complete
--------------------------------------
Loading subject 001
Data loaded
Subject 001 complete
--------------------------------------


In [6]:
X_train_tensors.shape

torch.Size([810, 1, 272, 1040])

In [7]:
X_train_tensors=X_train_tensors.cuda()
X_test_tensors=X_test_tensors.cuda()
y_train_tensors=y_train_tensors.cuda()
y_test_tensors=y_test_tensors.cuda()
print('done')

done


## band power 

In [19]:
X_train_numpy = X_train_tensors.cpu().numpy()
X_test_numpy = X_test_tensors.cpu().numpy()

X = np.swapaxes(X_train_numpy, 2, -1).squeeze()
data = X[X.shape[0]-1, 70, :]
psd_mne, freqs_mne = psd_array_welch(data, 100, 1., 70., n_per_seg=None,
                          n_overlap=0, n_jobs=1)
for low, high in [(0.5, 4), (4, 8), (8, 10), (10, 12), (12, 30),
                  (30, 70)]:
    print("processing bands (low, high) : ({},{})".format(low, high))
    # Find intersecting values in frequency vector
    idx_delta = np.logical_and(freqs_mne >= low, freqs_mne <= high)
      # Frequency resolution
    freq_res = freqs_mne[1] - freqs_mne[0]  # = 1 / 4 = 0.25

    # Compute the absolute power by approximating the area under the curve
    power = simps(psd_mne[idx_delta], dx=freq_res)
    print('Absolute power: {:.4f} uV^2'.format(power))
    
    total_power = simps(psd_mne, dx=freq_res)
    rel_power = power / total_power
    
    print('Relative power: {:.4f}'.format(rel_power))

Effective window size : 2.560 (s)
processing bands (low, high) : (0.5,4)
Absolute power: 0.3645 uV^2
Relative power: 0.4691
processing bands (low, high) : (4,8)
Absolute power: 0.0309 uV^2
Relative power: 0.0398
processing bands (low, high) : (8,10)
Absolute power: 0.0160 uV^2
Relative power: 0.0206
processing bands (low, high) : (10,12)
Absolute power: 0.0105 uV^2
Relative power: 0.0135
processing bands (low, high) : (12,30)
Absolute power: 0.2656 uV^2
Relative power: 0.3418
processing bands (low, high) : (30,70)
Absolute power: 0.0491 uV^2
Relative power: 0.0632


In [9]:
X_train_bp = np.squeeze(X_train_numpy, axis=1)
# X_train_bp = X_train_bp[: :, :, :]
X_train_bp = standard_scaling_sklearn(X_train_bp)
X_test_bp = np.squeeze(X_test_numpy, axis=1)
# X_train_bp = X_train_bp[: :, :, :]
X_test_bp = standard_scaling_sklearn(X_test_bp)
bands = [(1, 4), (4, 8), (8, 10), (10, 13), (13, 30), (30, 70)]
bp_train = bandpower_multi_bands(X_train_bp, fs=100.0, bands=bands, relative=True)
bp_test = bandpower_multi_bands(X_test_bp, fs=100.0, bands=bands, relative=True)
bp_train_tensor = torch.Tensor(bp_train).cuda()
bp_test_tensor = torch.Tensor(bp_test).cuda()

Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective window size : 2.560 (s)
Effective wind

In [10]:
bp_train_tensor.shape

torch.Size([810, 272, 6])

## Parms

In [11]:
CRED    = '\33[31m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CBLUE   = '\33[34m'

## Functions

In [12]:
class ChannelPool(nn.Module):

    def forward(self, x):
        return torch.cat((torch.max(x, 1)[0].unsqueeze(1),
                          torch.mean(x, 1).unsqueeze(1)), dim=1)

In [13]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int):
                How long to wait after last time validation loss improved.
                Default: 7
            verbose (bool):
                If True, prints a message for each validation loss improvement.
                Default: False
            delta (float):
                Minimum change in the monitored quantity to qualify as an improvement.
                Default: 0
            path (str):
                Path for the checkpoint to be saved to.
                Default: 'checkpoint.pt'
            trace_func (function):
                trace print function.
                Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.4f} --> {val_loss:.4f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [14]:
class SpatialAttention(nn.Module):
    def __init__(self):
        super(SpatialAttention, self).__init__()
        self.compress = ChannelPool()
        self.spatialAttention = nn.Sequential(
            nn.Conv2d(2, 1, 7, 7, padding=3), #padding = (7-1)/2
            )

    def forward(self, x):
        # print('x',x.shape)
        x = self.compress(x)
        x = self.spatialAttention(x)
        # scale = F.sigmoid(x)
        scale = torch.sigmoid(x)
        # print('scale',scale.shape)
        
        return x * scale


In [15]:
class Flatten_MEG(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

In [16]:
class ChannelAttention(nn.Module):
    """
            Implementation of a channel attention module.
        """
    class Showsize(nn.Module):
        def __init__(self):
            super(ChannelAttention.Showsize, self).__init__()
        def forward(self, x):
            # print(x.shape)
            return x

    def __init__(self, shape, reduction_factor=16):

        super(ChannelAttention, self).__init__()

        _, in_channel, h, w = shape
        self.mlp = nn.Sequential(
            # self.Showsize(),
            Flatten_MEG(),
            # self.Showsize(),
            nn.Linear(in_channel, in_channel // reduction_factor),
            nn.ReLU(),
            nn.Linear(in_channel // reduction_factor, in_channel),
        )
        # self.avg = nn.AvgPool2d(kernel_size=(h, w), stride=(h, w))
        # self.max = nn.MaxPool2d(kernel_size=(h, w), stride=(h, w))

    def forward(self, x):
        # print('x', x.shape)
        # avg = self.avg(x)
        # max = self.max(x)
        avg_pool = F.avg_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
        max_pool = F.max_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
        sum = self.mlp(avg_pool) + self.mlp(max_pool)
        # print(sum.shape)
        scale = (
            torch.sigmoid(sum)
            .unsqueeze(2)
            .unsqueeze(3)
            .expand_as(x)
        )

        return x * scale


In [17]:
class Concatenate(nn.Module):
    def __init__(self):
        super(Concatenate, self).__init__()

    def forward(self, x, bp):

        # min_ = x.min(1, keepdim=True)[0]
        # if min_[0] < 0:
        #     x = x + min_
        # else:
        #     x = x - min_
        # x = x / x.max()
        x = x.view(x.shape[0], -1)
        bp = bp.view(bp.shape[0], -1)
        x = torch.cat([x, bp], -1)

        return x

In [18]:
def GETcorrectnumber(loader, printcolor):
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for i in range(num_classes)]
        n_class_samples = [0 for i in range(num_classes)]
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = rpsmnet(inputs)
            optimizer.step()
            _, predicted = torch.max(outputs, 1)
            n_samples += labels.size(0)
            # n_correct += (predicted == labels).sum().item()
            for k in range(predicted.shape[0]):
                if predicted[k]==labels[k]:
                    n_correct +=1
            # for i in range(num_classes): # accuracy for each class
            #     label = labels[i]
            #     pred = predicted[i]
            #     if (label == pred):
            #         n_class_correct[i] += 1
            #     n_class_samples[i] += 1
        acc = 100.0 * n_correct / n_samples
        print(printcolor+f'[{epoch + 1}] t accuracy： {acc}%'+printcolor)
    return acc

In [19]:
class L1(torch.nn.Module):
    def __init__(self, module, weight_decay):
        super().__init__()
        self.module = module
        self.weight_decay = weight_decay

        # Backward hook is registered on the specified module
        self.hook = self.module.register_full_backward_hook(self._weight_decay_hook)

    # Not dependent on backprop incoming values, placeholder
    def _weight_decay_hook(self, *_):
        for param in self.module.parameters():
            # If there is no gradient or it was zeroed out
            # Zeroed out using optimizer.zero_grad() usually
            # Turn on if needed with grad accumulation/more safer way
            # if param.grad is None or torch.all(param.grad == 0.0):

            # Apply regularization on it
            param.grad = self.regularize(param)

    def regularize(self, parameter):
        # L1 regularization formula
        return self.weight_decay * torch.sign(parameter.data)

    def forward(self, *args, **kwargs):
        # Simply forward and args and kwargs to module
        return self.module(*args, **kwargs)

## RPS_Mnet

In [83]:
class RPS_MNet(nn.Module):
    """
        Model inspired by [Aoe at al., 10.1038/s41598-019-41500-x] integrated with bandpower.
    """
    class Showsize(nn.Module):
        def __init__(self):
            super(RPS_MNet.Showsize, self).__init__()
        def forward(self, x):
            # print(x.shape)
            return x

    def __init__(self, n_times):
        """
        Args:
            n_times (int):
                n_times dimension of the input data.
        """
        super(RPS_MNet, self).__init__()
        # if n_times == 501:  # TODO automatic n_times
        #     self.n_times = 12
        # elif n_times == 601:
        #     self.n_times = 18
        # elif n_times == 701:
        #     self.n_times = 24
        # else:
        #     raise ValueError(
        #         "Network can work only with n_times = 501, 601, 701 "
        #         "(epoch duration of 1., 1.2, 1.4 sec),"
        #         " got instead {}".format(n_times)
        #     )
        self.n_times = n_times
        self.spatial = nn.Sequential(
            nn.Conv2d(1, 32, stride=(1, 1), kernel_size=(272,64), bias=True), #kernel size 204, 64
            nn.ReLU(),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, stride=(1, 1), kernel_size=(1, 16), bias=True), # kernel size 1,16
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            # nn.BatchNorm2d(64),
        )

        self.temporal = nn.Sequential(
            nn.Conv2d(1, 32, stride=(1, 1), kernel_size=(8, 8), bias=True),
            nn.ReLU(),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, stride=(1, 1), kernel_size=(8, 8), bias=True),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(5, 3), stride=(5, 3)),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, stride=(1, 1), kernel_size=(1, 4), bias=True),
            nn.ReLU(),
            # nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, stride=(1, 1), kernel_size=(1, 4), bias=True), #conv6
            nn.ReLU(),
            # nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Conv2d(64, 128, stride=(1, 1), kernel_size=(1, 2), bias=True),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, stride=(1, 1), kernel_size=(1, 2), bias=True),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Conv2d(128, 256, stride=(1, 1), kernel_size=(1, 2), bias=True),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, stride=(1, 1), kernel_size=(1, 2), bias=True), #conv10
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
        )

        self.attention1 = nn.Sequential(
            ChannelAttention([None, 256, 26, self.n_times]),
        )

        self.attention2 = nn.Sequential(
            SpatialAttention(),
        )

        self.concatenate = Concatenate()

        self.flatten = Flatten_MEG()

        self.ff1 = nn.Sequential(
            # nn.Linear(256 * 26 * self.n_times + 272 * 6, 1024),
            nn.Linear(4 , 1024),
            # nn.BatchNorm1d(num_features=1024),
            nn.ReLU(),
            self.Showsize(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=1024),
            # nn.ReLU(),
            nn.Dropout(0.3),
            self.Showsize(),
        )
        self.ff2 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(2656 , 14),
        )
        self.softmax = nn.Softmax(dim =1)

    def forward(self, x, pb = pb):
        x = self.spatial(x)
        x = torch.transpose(x, 1, 2)
        x = self.temporal(x)
        x = self.attention1(x)
        x = self.attention2(x)
        x = self.ff1(self.flatten(x))
        x = self.concatenate(x, pb)
        x = self.ff2(x)
        x = self.softmax(x)

        return x.squeeze(1)

In [84]:
# class LSTM1(nn.Module):
#     def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
#         super(LSTM1, self).__init__()
#         self.num_classes = num_classes #number of classes
#         self.num_layers = num_layers #number of layers
#         self.input_size = input_size #input size
#         self.hidden_size = hidden_size #hidden state
#         self.seq_length = seq_length #sequence length

#         self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
#                           num_layers=num_layers, batch_first=True) #lstm
#         self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
#         self.fc = nn.Linear(128, num_classes) #fully connected last layer

#         self.relu = nn.ReLU()
    
#     def forward(self,x):
#         h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
#         c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state
#         # Propagate input through LSTM
#         output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
#         hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
#         out = self.relu(hn)
#         out = self.fc_1(out) #first Dense
#         out = self.relu(out) #relu
#         out = self.fc(out) #Final Output
#         return out

In [85]:
# rpsmnet = LSTM1(num_classes=14, input_size=280, hidden_size=10, num_layers=1, seq_length= 280).to(device) #our lstm class 
# print(rpsmnet)
pb = torch.rand(64,272,6).cuda()

In [86]:
# rpsmnet = AlexNet().cuda()
# print(rpsmnet)

In [87]:
rpsmnet = RPS_MNet(30).cuda()
print(rpsmnet)

RPS_MNet(
  (spatial): Sequential(
    (0): Conv2d(1, 32, kernel_size=(272, 64), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(1, 16), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (temporal): Sequential(
    (0): Conv2d(1, 32, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(8, 8), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(5, 3), stride=(5, 3), padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(1, 4), stride=(1, 1))
    (6): ReLU()
    (7): Conv2d(64, 64, kernel_size=(1, 4), stride=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(64, 128, kernel_size=(1, 2), stride=(1, 1))
    (11): ReLU()
    (12): Dropout2d(p=0.3, inplace=False)
    (13): Conv2d(128, 128, kernel_size=(1, 2), stride=(1, 1))
    (14): ReLU()
  

In [88]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(rpsmnet)

+----------------------------------------+------------+
|                Modules                 | Parameters |
+----------------------------------------+------------+
|            spatial.0.weight            |   557056   |
|             spatial.0.bias             |     32     |
|            spatial.2.weight            |   32768    |
|             spatial.2.bias             |     64     |
|           temporal.0.weight            |    2048    |
|            temporal.0.bias             |     32     |
|           temporal.2.weight            |   65536    |
|            temporal.2.bias             |     32     |
|           temporal.5.weight            |    8192    |
|            temporal.5.bias             |     64     |
|           temporal.7.weight            |   16384    |
|            temporal.7.bias             |     64     |
|           temporal.10.weight           |   16384    |
|            temporal.10.bias            |    128     |
|           temporal.13.weight           |   327

2031329

In [89]:
# # !pip uninstall torchviz
# sudo apt-get install graphviz
import hiddenlayer as hl

transforms = [ hl.transforms.Prune('Constant') ] # Removes Constant nodes from graph.

graph = hl.build_graph(rpsmnet, torch.zeros(64,1,272,800).cuda())
graph.theme = hl.graph.THEMES['blue'].copy()
graph.save('rnn_hiddenlayer', format='png')

TypeError: _jit_pass_onnx_unpack_quantized_weights(): incompatible function arguments. The following argument types are supported:
    1. (arg0: torch::jit::Graph, arg1: Dict[str, IValue], arg2: bool) -> Dict[str, IValue]

Invoked with: graph(%input.1 : Float(64, 1, 272, 800, strides=[217600, 217600, 800, 1], requires_grad=0, device=cuda:0),
      %1 : Float(32, 1, 272, 64, strides=[17408, 17408, 64, 1], requires_grad=1, device=cuda:0),
      %2 : Float(32, strides=[1], requires_grad=1, device=cuda:0),
      %3 : Float(64, 32, 1, 16, strides=[512, 16, 16, 1], requires_grad=1, device=cuda:0),
      %4 : Float(64, strides=[1], requires_grad=1, device=cuda:0),
      %5 : Float(32, 1, 8, 8, strides=[64, 64, 8, 1], requires_grad=1, device=cuda:0),
      %6 : Float(32, strides=[1], requires_grad=1, device=cuda:0),
      %7 : Float(32, 32, 8, 8, strides=[2048, 64, 8, 1], requires_grad=1, device=cuda:0),
      %8 : Float(32, strides=[1], requires_grad=1, device=cuda:0),
      %9 : Float(64, 32

In [90]:
rpsmnet = RPS_MNet(30).cuda()

from torchviz import make_dot
x = torch.randn(64, 1, 272, 800).requires_grad_(True).cuda() # 定义一个网络的输入值
y = rpsmnet(x)    # 获取网络的预测值
# y = y.cuda()
MyConvNetVis = make_dot(y)#, params=dict(list(rpsmnet.named_parameters()) + [('x', x)]))
MyConvNetVis.format = "png"
# 指定文件生成的文件夹
MyConvNetVis.directory = "data"
# 生成文件
MyConvNetVis.view()

AttributeError: module 'graphviz.backend' has no attribute 'ENCODING'

In [94]:
!pip install torchviz==0.0.2

In [95]:
from torchviz import make_dot
x=torch.ones(10, requires_grad=True)

y=x**2
z=x**3
r=(y+z).sum()
make_dot(r).render("attached", format="png")

AttributeError: module 'graphviz.backend' has no attribute 'ENCODING'

## Train

In [58]:
BATCH_SIZE=64
num_epochs=1
train = Data.TensorDataset(X_train_tensors, y_train_tensors, bp_train_tensor)
test = Data.TensorDataset(X_test_tensors, y_test_tensors, bp_test_tensor)
train_loader = Data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = Data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

learning_rate = 0.0005
criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(rpsmnet.parameters(), lr=learning_rate)#, weight_decay=0.001)
# optimizer = torch.optim.SGD(rpsmnet.parameters(), lr=learning_rate, momentum = 0.9)

optimizer = torch.optim.SGD([{'params': rpsmnet.spatial.parameters()}, 
                             {'params': rpsmnet.temporal.parameters()},
                             {'params': rpsmnet.attention1.parameters()},
                             {'params': rpsmnet.attention2.parameters()},
                             {'params': rpsmnet.ff1.parameters()},
                             {'params': rpsmnet.concatenate.parameters(),'weight_decay': 0.0005},
                             {'params': rpsmnet.ff2.parameters(), 'weight_decay': 0.0005}], lr=learning_rate, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max')

In [59]:
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
summary_writer = SummaryWriter(f'./models/test')
# valid_loss = []
running_loss = 0
door_for_test = 1
rpsmnet.train()
for epoch in range(num_epochs):  # loop over the dataset multiple times
    t_loss = 0
    train_loss = []
    for i, data in enumerate(train_loader, 0):
#         get the inputs; data is a list of [inputs, labels]
        inputs, labels, bp = data #, bp
        # print(inputs.shape)
        inputs, labels, bp = inputs.to(device), labels.type(torch.LongTensor).to(device), bp.to(device)# , bp  , bp.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = rpsmnet(inputs,bp)
        # print(outputs)
        with torch.autocast('cuda'):
        # loss = criterion(outputs, torch.tensor(labels).cuda())
            loss = criterion(outputs, labels)
#         l2_lambda = 0.001
#         l2_reg = torch.tensor(0.).cuda()
#         for param in rpsmnet.parameters():
#             l2_reg += torch.norm(param)
#         loss += l2_lambda * l2_reg
        
        # l1_lambda = 0.001
        # l1_norm = sum(torch.linalg.norm(p, 1) for p in rpsmnet.parameters())
        # # l2_norm = sum(torch.linalg.norm(p, 2) for p in model.parameters())
        # loss = loss + l1_lambda * l1_norm
        loss.backward()
        optimizer.step()
        t_loss += loss.item()
        # print(t_loss)
        train_loss.append(np.mean(t_loss))
    
    
    if epoch % 1 == 0:
        # if i % (math.ceil(900*Split/BATCH_SIZE)-1) == 0 and i !=0:
        print(CRED+ f'[{epoch + 1}, {i + 1}] trainning loss: {train_loss[-1]}'+ CRED)
    summary_writer.add_scalar('train_loss', train_loss[-1], epoch)
    
    if door_for_test == 1:
        if epoch % 1 == 0:
            # rpsmnet.eval()
            valid_loss = []
            acc = []
            va_loss = 0
            for i, data in enumerate(test_loader, 0):
                i_list = []
                i_list.append(i)
                val_x, val_y, bp = data #, bp 
                val_x, val_y, bp = val_x.to(device), val_y.type(torch.LongTensor).to(device), bp.to(device)#, bp.to(device) #, bp
                Testoutput = rpsmnet(val_x, bp)# , bp
                # v_loss = criterion(Testoutput, val_y, torch.Tensor(Testoutput.size(0)).cuda().fill_(1.0))
                v_loss = criterion(Testoutput, val_y) #loss
                va_loss += v_loss.item()
                valid_loss.append(np.mean(va_loss))
                # if i = i_list[-1]:    # print every first
                _, predicted = torch.max(Testoutput,1)
                labels = val_y.cpu()
                predicted = predicted.cpu()
                mean_accuracy = accuracy_score(labels[labels != 99], predicted[predicted != 99])
                acc.append(np.mean(mean_accuracy))
            print(CGREEN+f'[{epoch + 1}, {i + 1}] valid_loss: {valid_loss[-1]} Acc: {acc[-1]}'+CGREEN)
            # rpsmnet.train()
        summary_writer.add_scalar('valid_loss', valid_loss[-1], epoch)
#     scheduler.step(acc[-1])

  # if epoch % 1 == 0:
  #   GETcorrectnumber(train_loader,CYELLOW)      
  #   GETcorrectnumber(test_loader,CBLUE)
  # aoemnet.train()
    early_stopping = EarlyStopping(patience=10, verbose=True)
    early_stopping(valid_loss[-1], rpsmnet)

    if early_stopping.early_stop:
        print("Early stopping!")
        break

print('Finished Training')

[1, 13] trainning loss: 34.30713725090027
[1, 2] valid_loss: 5.279666185379028 Acc: 0.038461538461538464
Validation loss decreased (inf --> 5.2797).  Saving model ...
Finished Training


In [ ]:
make_dot(outputs, params=dict(list(rpsmnet.named_parameters()))).render("rnn_torchviz", format="png")

In [ ]:
test1 = torch.rand(64, 256, 26, 30)
test2 = torch.rand(64, 256, 26, 1).expand_as(test1)

test3 = test1*test2
test3 = test3.squeeze(1)
print(test3.shape)

In [ ]:
test = inputs.squeeze(1).shape
print(test)

Confusion matrix and mean accuracy

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
confusion_matrix_test_loader = Data.DataLoader(test, batch_size = 90, shuffle = False)
with torch.no_grad():
    list_mean_accuracy = []
    for i, data in enumerate(confusion_matrix_test_loader, 0):
        inputs, labels, bp = data
        inputs, labels, bp = inputs.to(device), labels.type(torch.LongTensor).to(device), bp.to(device)
        # inputs, labels = data
        # inputs, labels = inputs.to(device), labels.type(torch.LongTensor).to(device)
        outputs = rpsmnet(inputs, bp)
        # outputs = rpsmnet(inputs)
#         optimizer.step()          
        _, predicted = torch.max(outputs,1)
        # predicted = torch.max(outputs)
        labels = labels.cpu()
        predicted = predicted.cpu()
        labels = labels.numpy()
        predicted = predicted.numpy()
        # mean_conf_mat = confusion_matrix(labels, predicted)
        # mean_accuracy = accuracy_score(labels[labels != 99], predicted[predicted != 99])
        # mean_conf_mat = mean_conf_mat.astype('float') / mean_conf_mat.sum(axis=1)  # normalise
        mean_conf_mat = confusion_matrix(labels, predicted)
        mean_accuracy = accuracy_score(labels[labels != 99], predicted[predicted != 99])
        mean_conf_mat = mean_conf_mat.astype('float') / mean_conf_mat.sum(axis=1) 
        list_mean_accuracy.append(mean_accuracy)
        print("Mean accuracy = {0}".format(mean_accuracy))
        ConfusionMatrixDisplay.from_predictions(labels, predicted)
        # plt.savefig('/content/drive/MyDrive/MT_ML_Decoding/Aversive_state_reactivation/notebooks/templates/save_folder/fig-{}.png'.format(session_id), dpi=600)
        plt.show()

In [ ]:
torch.max(Testoutput,1)

In [ ]:
!rm -rf /logs/ # clear logs
# if 'google.colab' in str(get_ipython()): # tensor board
%load_ext tensorboard  
# %tensorboard --logdir logs
%tensorboard --logdir=./models/test

## Test

In [ ]:
# class CNN(nn.Module):
#     def __init__(self):
#       # super(CNN, self)._init_()
#       super(CNN, self).__init__()
#       self.n_classes = 14
#       n_classes =14
#       self.conv1 = nn.Sequential(
#           nn.Conv2d(
#               in_channels=1,
#               out_channels=32,
#               kernel_size=3,
#               stride=1,
#               padding=1,
#           ),
#           nn.ReLU(),
#           nn.MaxPool2d(kernel_size=2,stride=2),
#       )
#       self.conv2 = nn.Sequential(
#           nn.Conv2d(32,64,3,1,1),
#           nn.ReLU(),
#           nn.MaxPool2d (2,2),
#       )
#       # self.fc = nn.Linear(64*7*7,128)
#       self.fc = nn.Linear(139264, 100)
#       self.out = nn.Linear(100,n_classes)
#       self.softmax = nn.Softmax()

#     def forward(self,x):
#       x=self.conv1(x)
#       x=self.conv2(x)
#       # x=x.view(x.size(0),-1)
#       x = torch.flatten(x, 1) # flatten all dimensioxns except batch
#       x=self.fc(x)
#       x=self.out(x)      
#       # output=self.out(x)
#       # return output, x
#       # x=self.softmax(x)
#       return x